In [30]:
import numpy as np

import sys
module_path = '/home/diplomski-rad/consensus-net/src/python/dataset/'
if module_path not in sys.path:
    print('Adding dataset module.')
    sys.path.append(module_path)

import dataset

## Create dataset with neighbourhood

In [2]:
X_paths = [
    '/home/diplomski-rad/blade/pb/racon-hax-pileups/e-coli-NCTC86-racon-MSA/pileups-X-0.npy',
    '/home/diplomski-rad/blade/pb/racon-hax-pileups/m-morgani-NCTC235-racon-MSA/pileups-X-0.npy',
    '/home/diplomski-rad/blade/pb/racon-hax-pileups/s-enterica-NCTC92-racon-MSA/pileups-X-0.npy',
    '/home/diplomski-rad/blade/pb/racon-hax-pileups/s-enterica-NCTC129-racon-MSA/pileups-X-0.npy'
]
y_paths = [
    '/home/diplomski-rad/blade/pb/racon-hax-pileups/e-coli-NCTC86-racon-MSA/pileups-y-0.npy',
    '/home/diplomski-rad/blade/pb/racon-hax-pileups/m-morgani-NCTC235-racon-MSA/pileups-y-0.npy',
    '/home/diplomski-rad/blade/pb/racon-hax-pileups/s-enterica-NCTC92-racon-MSA/pileups-y-0.npy',
    '/home/diplomski-rad/blade/pb/racon-hax-pileups/s-enterica-NCTC129-racon-MSA/pileups-y-0.npy'
]
neighbourhood_size = 20
save_directory_path = './'

X, y = dataset.create_dataset_with_neighbourhood(neighbourhood_size, mode='training',
    X_paths=X_paths, y_paths=y_paths)

X, y = X[0], y[0]

  0% (1922 of 31260819) |                | Elapsed Time: 0:00:00 ETA:   0:27:07

Creating dataset with neighbourhood ...


 26% (8433625 of 31260819) |#     | Elapsed Time: 0:07:48 ETA:  1 day, 13:39:58

Creating dataset with neighbourhood ...


 47% (14935299 of 31260819) |##    | Elapsed Time: 0:13:58 ETA:  1 day, 0:17:38

Creating dataset with neighbourhood ...


 74% (23380080 of 31260819) |########    | Elapsed Time: 0:22:04 ETA:  12:36:20

Creating dataset with neighbourhood ...


100% (31260819 of 31260819) |############| Elapsed Time: 0:29:31 Time:  0:29:31


## Split train and val dataset

In [3]:
from sklearn.model_selection import train_test_split

validation_size = 0.1

X_train, X_validate, y_train, y_validate = train_test_split(
            X, y, test_size=validation_size)

In [4]:
np.save('./dataset-n20-X-train', X_train)
np.save('./dataset-n20-X-validate', X_validate)
np.save('./dataset-n20-y-train', y_train)
np.save('./dataset-n20-y-validate', y_validate)

## Prepare for finetune

In [37]:
X_train = np.load('./dataset-n20-X-train.npy')
y_train = np.load('./dataset-n20-y-train.npy')

In [38]:
class_A = np.array([1, 0, 0, 0, 0, 0])
class_A_indices = np.where(np.all(y_train == class_A, axis=1))[0]

class_C = np.array([0, 1, 0, 0, 0, 0])
class_C_indices = np.where(np.all(y_train == class_C, axis=1))[0]

class_G = np.array([0, 0, 1, 0, 0, 0])
class_G_indices = np.where(np.all(y_train == class_G, axis=1))[0]

class_T = np.array([0, 0, 0, 1, 0, 0])
class_T_indices = np.where(np.all(y_train == class_T, axis=1))[0]

class_I = np.array([0, 0, 0, 0, 1, 0])
class_I_indices = np.where(np.all(y_train == class_I, axis=1))[0]

class_D = np.array([0, 0, 0, 0, 0, 1])
class_D_indices = np.where(np.all(y_train == class_D, axis=1))[0]

In [39]:
class_A_permuted_indices = np.random.permutation(class_A_indices)
class_C_permuted_indices = np.random.permutation(class_C_indices)
class_G_permuted_indices = np.random.permutation(class_G_indices)
class_T_permuted_indices = np.random.permutation(class_T_indices)
class_I_permuted_indices = np.random.permutation(class_I_indices)
class_D_permuted_indices = np.random.permutation(class_D_indices)

In [40]:
X_train_finetune_A = X_train[class_A_permuted_indices[:200000]]
X_train_finetune_C = X_train[class_C_permuted_indices[:200000]]
X_train_finetune_G = X_train[class_G_permuted_indices[:200000]]
X_train_finetune_T = X_train[class_T_permuted_indices[:200000]]
X_train_finetune_I = X_train[class_I_permuted_indices[:200000]]
X_train_finetune_D = X_train[class_D_permuted_indices] # all deletions

y_train_finetune_A = y_train[class_A_permuted_indices[:200000]]
y_train_finetune_C = y_train[class_C_permuted_indices[:200000]]
y_train_finetune_G = y_train[class_G_permuted_indices[:200000]]
y_train_finetune_T = y_train[class_T_permuted_indices[:200000]]
y_train_finetune_I = y_train[class_I_permuted_indices[:200000]]
y_train_finetune_D = y_train[class_D_permuted_indices]

In [41]:
X_train_finetune = np.concatenate(
    (X_train_finetune_A, X_train_finetune_C, X_train_finetune_G, X_train_finetune_T, X_train_finetune_I, X_train_finetune_D),
    axis=0
)
y_train_finetune = np.concatenate(
    (y_train_finetune_A, y_train_finetune_C, y_train_finetune_G, y_train_finetune_T, y_train_finetune_I, y_train_finetune_D),
    axis=0
)
# Permute to shuffle data.
X_train_finetune = X_train_finetune[np.random.permutation(np.arange(X_train_finetune.shape[0]))]
y_train_finetune = y_train_finetune[np.random.permutation(np.arange(y_train_finetune.shape[0]))]

In [42]:
np.save('./dataset-n20-X-train-finetune-3.npy', X_train_finetune,)
np.save('./dataset-n20-y-train-finetune-3.npy', y_train_finetune)